Reference: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/util/nest.py

In [1]:
import collections
from tensorflow.python.util import nest

### _sequence_like
- convert type of second argument as type of first argument

#### tuple, list

In [2]:
nest._sequence_like([1,2,3], (1,2,3))

[1, 2, 3]

In [3]:
nest._sequence_like((1,2,3), [1,2])

(1, 2)

#### namedtuple

In [4]:
test_ntuple = collections.namedtuple('test_namedtuple', ['x', 'y', 'z'])(1,2,3)

In [5]:
test_ntuple

test_namedtuple(x=1, y=2, z=3)

In [6]:
nest._sequence_like(test_ntuple, (1,2,3))

test_namedtuple(x=1, y=2, z=3)

### _yield_flat_nest
- yield a generator that flattens given nested sequence

In [7]:
nest._yield_flat_nest([[1,2,3]])

<generator object _yield_flat_nest at 0x119960888>

In [8]:
list(nest._yield_flat_nest([[1,2,3]]))

[1, 2, 3]

### is_sequence
- check if input is sequence(collections.Sequence) except string

In [9]:
nest.is_sequence(list())

True

In [10]:
nest.is_sequence(test_ntuple)

True

In [11]:
nest.is_sequence('hello')

False

### flatten
- flatten given nested sequence

In [12]:
nest.flatten([[1,2,3],[2,3,[4]]])

[1, 2, 3, 2, 3, 4]

### _recursive_assert_same_structure
- helper function for `assert_same_structure`

In [13]:
nest._recursive_assert_same_structure([[1,2,3], [2,3,[4]]], [[1,2,3], [3,4,[5]]])

In [14]:
try:
    nest._recursive_assert_same_structure([[1,2,3], [2,3,[4]]], [1,2,3,3,4,5])
except ValueError as e:
    print(e)

The two structures don't have the same nested structure. First structure: [1, 2, 3], second structure: 1.


### assert_same_structure
- validates two nested sequence have same structure

In [15]:
try:
    nest.assert_same_structure([[1,2,3], [2,3,[4]]], [[1,2,3], [3,4,5]])
except ValueError as e:
    print(e)

The two structures don't have the same nested structure. First structure: [4], second structure: 5.


### flatten_dict_items
- flatten nested dictionary

In [16]:
nest.flatten_dict_items(
    {
        ('a','b',('c','d')): (1, 2, (3, 4))
    }
)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

### _packed_nest_with_indices
- helper function for `pack_sequence_as`

In [17]:
nest._packed_nest_with_indices(
    structure=(1,2,(3),()),
    flat=[1,2,3],
    index=0)

(3, [1, 2, 3, ()])

### pack_sequence_as
- pack flat sequence in the structure of first argument

In [18]:
nest.pack_sequence_as(
    [1,2,3,[4,[5,6]]],
    ['a','b',3,4,5,7])

['a', 'b', 3, [4, [5, 7]]]

### map_structure
- `map` wrapper for neseted sequence

In [19]:
nest.map_structure(
    lambda x, y: x + y, 3,4)

7

In [20]:
nest.map_structure(
    lambda x, y: x+y,
    (((1, 2), 3), 4, (5, 6)), (((7, 8), 9), 10, (11, 12))
    )

(((8, 10), 12), 14, (16, 18))

### _yield_flat_up_to
- helper function for `assert_shallow_structure`

In [21]:
nest._yield_flat_up_to(
    ['a', 'b'],
    ['c', ['d', 'e']])

<generator object _yield_flat_up_to at 0x1199606d0>

In [22]:
_y = nest._yield_flat_up_to(
    ['a', 'b'],
    ['c', ['d', 'e']])
next(_y)

'c'

In [23]:
next(_y)

['d', 'e']

### assert_shallow_structure
- validates two nested sequence have same 'shallow structure'
- don't raise error if each element of first sequence can correspond to leaf node of second sequence

In [24]:
nest.assert_shallow_structure(
    ['a', 'b'],
    ['c', ['d', 'e']])

In [25]:
try:
    nest.assert_shallow_structure(
    ['a', 'b'],
    ['c', ['d', 'e'], 'f'])
except ValueError as e:
    print(e)

The two structures don't have the same sequence length. Input structure has length 3, while shallow structure has length 2.


### flatten_up_to
- flatten input_tree up to shallow_tree's structure

In [26]:
nest.flatten_up_to(
    shallow_tree=[[True, True],
                  [False, True]],
    input_tree=[[[2, 2], [3, 3]],
                [[4, 9], [5, 5]]]
)

[[2, 2], [3, 3], [4, 9], [5, 5]]

In [27]:
nest.flatten_up_to(
    [[True, True], [False, False]],
    [[True, True], [False, False]])

[True, True, False, False]

In [28]:
nest.flatten_up_to(0, [0, 1, 2])

[[0, 1, 2]]

In [29]:
nest.flatten_up_to([0, [1, 2]], [[0], [1, [2]]])

[[0], 1, [2]]

In [30]:
try:
    nest.flatten_up_to([[0,0], [1,2,3]], [[0], [1]])
except ValueError as e:
    print(e)

The two structures don't have the same sequence length. Input structure has length 1, while shallow structure has length 2.


In [31]:
try:
    nest.flatten_up_to([0, 1, 2], 0)
except TypeError as e:
    print(e)

If shallow structure is a sequence, input must also be a sequence. Input has type: <class 'int'>.


### map_structure_up_to

In [32]:
ab_tuple = collections.namedtuple("ab_tuple", "a, b")
op_tuple = collections.namedtuple("op_tuple", "add, mul")
inp_vals = ab_tuple(a=2, b=3)
inp_ops = ab_tuple(a=op_tuple(add=1, mul=2), b=op_tuple(add=2, mul=3))

In [33]:
inp_vals

ab_tuple(a=2, b=3)

In [34]:
inp_ops

ab_tuple(a=op_tuple(add=1, mul=2), b=op_tuple(add=2, mul=3))

In [35]:
nest.map_structure_up_to(
    inp_vals, # shallow_tree
    lambda val, ops: (val + ops.add) * ops.mul, # func
    inp_vals, inp_ops) # *inputs

# a=(2+1)*2, b=(3+2)*3

ab_tuple(a=6, b=15)

In [36]:
nest.map_structure_up_to(
    ['evens', ['odds', 'primes']],
    lambda name, sec: "first_{}_{}".format(len(sec), name),
    ['evens', ['odds', 'primes']], [[2, 4, 6, 8], [[1, 3, 5, 7, 9], [3, 5, 7]]])

# [first_len([2,4,6,8])_evens ... ]

['first_4_evens', ['first_5_odds', 'first_3_primes']]